In [1]:
# LangChain V0.3版本
!pip install langchain-community \
        langgraph \
        langchain-openai \
        tavily-python \
        langgraph-checkpoint-sqlite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: lang

In [8]:
# 将使用Tavily（搜索引擎）作为工具。要使用它，你需要获取并设置API密钥：
# Tavily's Search API地址 https://tavily.com/
import os
os.environ["TAVILY_API_KEY"] = "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" # 将 API key 设置为环境变量


In [9]:
# 导入相关的 LangChain 功能模块
# 从 langchain_openai 库导入 ChatOpenAI 和 OpenAIEmbeddings，用于使用 OpenAI 的聊天模型和嵌入模型
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults  # 导入 Tavily 搜索引擎工具
from langchain_core.messages import HumanMessage  # 导入 HumanMessage 类型，用于表示用户输入
from langgraph.checkpoint.memory import MemorySaver  # 导入 MemorySaver，用于内存存储对话历史
from langgraph.prebuilt import create_react_agent  # 导入 create_react_agent 函数，用于创建 React Agent

In [11]:
# 创建 Agent 的组件

# 初始化内存存储器，用于保存对话历史 (可选，用于有状态 Agent)
memory = MemorySaver()

# Create a ChatOpenAI model
# OpenAI API调用（代理方式）
model = ChatOpenAI(
    api_key="sk-paSHgQoVeKag1rou9d81Fa2f534940C1Ba394f02C45aF3D2",
    base_url="https://vip.apiyi.com/v1"
)


# 初始化 Tavily 搜索引擎工具，设置最大搜索结果数为 2
search = TavilySearchResults(max_results=2)
# 将 Tavily 搜索引擎工具放入工具列表，Agent 可以使用这些工具
tools = [search]

# 使用 `create_react_agent` 函数创建 Agent 执行器
# 该函数会自动将模型和工具绑定，并使用 ReAct 框架构建 Agent
# checkpointer=memory 参数用于启用对话记忆功能
agent_executor = create_react_agent(model, tools, checkpointer=memory)


# ---  首次对话：打招呼并告知所在地 ---
# 配置对话线程 ID，用于记忆功能，相同 thread_id 共享对话历史
config = {"configurable": {"thread_id": "abc123"}}
print("--- 首次对话 ---")
# 使用 stream 方法进行流式输出，逐步显示 Agent 的响应
for chunk in agent_executor.stream(
    # 构造用户输入消息 HumanMessage -  用户用中文打招呼，并告知住在北京
    {"messages": [HumanMessage(content="你好！我是小明，我住在北京。")]}, config
):
    # 打印每个 chunk 的输出，展示 Agent 的思考过程
    print(chunk)
    print("----")


# ---  第二次对话：询问天气 ---
print("\n--- 第二次对话 ---")
for chunk in agent_executor.stream(
    # 再次使用相同的 thread_id，Agent 将记住之前的对话 -  用户用中文询问天气
    {"messages": [HumanMessage(content="我住的地方天气怎么样？")]}, config
):
    print(chunk)
    print("----")


# ---  第三次对话： 询问名字 ---
print("\n--- 第三次对话 (验证记忆) ---")
for chunk in agent_executor.stream(
    # 继续使用相同的 thread_id -  用户用中文询问名字
    {"messages": [HumanMessage(content="请问我叫什么名字？")]}, config
):
    print(chunk)
    print("----")


# ---  新的对话：更换线程 ID，验证无记忆 ---
# 使用新的 thread_id，开启新的对话
config_new_thread = {"configurable": {"thread_id": "xyz123"}}
print("\n--- 新的对话 (无记忆) ---")
for chunk in agent_executor.stream(
    # 用户在新对话线程中，用中文询问名字
    {"messages": [HumanMessage(content="我叫什么名字？")]}, config_new_thread
):
    print(chunk)
    print("----")


print("\n---  代码执行完成  ---")

--- 首次对话 ---
{'agent': {'messages': [AIMessage(content='你好，小明！很高兴认识你。北京是个充满历史和文化的地方，你最喜欢那里哪个景点呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 93, 'total_tokens': 143, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5f9a334b-e46c-4afc-bebd-f1db22860bb9-0', usage_metadata={'input_tokens': 93, 'output_tokens': 50, 'total_tokens': 143, 'input_token_details': {}, 'output_token_details': {}})]}}
----

--- 第二次对话 ---
{'agent': {'messages': [AIMessage(content='我无法提供实时天气信息，但你可以通过天气应用或网站查询北京的最新天气情况。一般来说，北京冬季寒冷干燥，夏季炎热潮湿。你今天打算出去吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 165, 'total_tokens': 248, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish

In [13]:
# 导入 getpass 和 os 模块
# Imports getpass and os modules
import getpass
import os

# 检查是否已设置 Tavily API 密钥，如果未设置，则提示用户输入
# Checks if Tavily API key is set, prompts user for input if not
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

# （可选）设置 LangSmith 以获得最佳的可观测性
# (Optional) Set up LangSmith for best-in-class observability
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

# 从 langchain_community.tools 导入 TavilySearchResults
# Imports TavilySearchResults from langchain_community.tools
from langchain_community.tools import TavilySearchResults

# 实例化 TavilySearchResults 工具
# Instantiates TavilySearchResults tool
tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",                   # overwrite default tool name
    # description="...",            # overwrite default tool description
    # args_schema=...,              # overwrite default args_schema: BaseModel
)

# 直接调用工具，传入查询参数
# Directly invokes the tool with query argument
tool.invoke({"query": "What happened at the last wimbledon"})

# 创建一个模型生成的 ToolCall 用于演示目的
# Creates a model-generated ToolCall for demo purposes
model_generated_tool_call = {
    "args": {"query": "euro 2024 host nation"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
# 调用工具并获取 ToolMessage
# Invokes the tool and gets ToolMessage
tool_msg = tool.invoke(model_generated_tool_call)

# 打印 ToolMessage 内容的前 400 个字符 (JSON 字符串形式)
# Prints the first 400 characters of ToolMessage content (JSON string format)
print(tool_msg.content[:400])

# 检查 ToolMessage artifact 中各项的类型
# Checks the type of each item in ToolMessage artifact
{k: type(v) for k, v in tool_msg.artifact.items()}

# 导入 json 模块
# Imports json module
import json

# 缩写 artifact 中的结果，并打印 JSON 格式的 artifact 信息 (缩写后的字符串)
# Abbreviates results in artifact and prints artifact info in JSON format (abbreviated strings)
print(json.dumps({k: str(v)[:200] for k, v in tool_msg.artifact.items()}, indent=2))

# 安装 langchain[openai] 包
# Installs langchain[openai] package
# %pip install -qU "langchain[openai]" # 注释掉，避免重复安装，如果需要使用chaining功能再取消注释

# 再次检查并设置 OpenAI API 密钥，如果未设置，则提示用户输入
# Checks and sets OpenAI API key again, prompts user for input if not
# import getpass # 注释掉，避免重复导入
# import os # 注释掉，避免重复导入

# if not os.environ.get("OPENAI_API_KEY"): # 注释掉，避免重复设置，如果需要使用chaining功能再取消注释
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ") # 注释掉，避免重复设置，如果需要使用chaining功能再取消注释

# 从 langchain.chat_models 导入 init_chat_model
# Imports init_chat_model from langchain.chat_models
from langchain.chat_models import ChatOpenAI # 修改为ChatOpenAI，init_chat_model 在新版本中可能已弃用

# 初始化聊天模型 (使用 gpt-4o-mini 模型， provider 为 openai)
# Initializes chat model (using gpt-4o-mini model, provider is openai)
llm = ChatOpenAI(
    api_key="sk-paSHgQoVeKag1rou9d81Fa2f534940C1Ba394f02C45aF3D2",
    base_url="https://vip.apiyi.com/v1",
    model_name="gpt-4o-mini"
)
# llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=os.environ.get("OPENAI_API_KEY")) # 修改为 ChatOpenAI 实例化方式，并显式传入API key

# 导入 datetime 模块
# Imports datetime module
import datetime

# 从 langchain_core.prompts 导入 ChatPromptTemplate
# Imports ChatPromptTemplate from langchain_core.prompts
from langchain_core.prompts import ChatPromptTemplate
# 从 langchain_core.runnables 导入 RunnableConfig 和 chain
# Imports RunnableConfig and chain from langchain_core.runnables
from langchain_core.runnables import RunnableConfig, chain

# 获取今天的日期并格式化
# Gets today's date and formats it
today = datetime.datetime.today().strftime("%D")
# 定义提示模板
# Defines prompt template
prompt = ChatPromptTemplate(
    [
        ("system", f"You are a helpful assistant. The date today is {today}."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# 将工具绑定到语言模型，指定 tool_choice 以强制模型调用该工具
# Binds tool to language model, specifies tool_choice to force model to call this tool
llm_with_tools = llm.bind_tools([tool])

# 创建 LLM 链
# Creates LLM chain
llm_chain = prompt | llm_with_tools

# 定义工具链函数
# Defines tool chain function
@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 调用工具链，查询 “who won the last womens singles wimbledon”
# Invokes tool chain, querying "who won the last womens singles wimbledon"
tool_chain.invoke("who won the last womens singles wimbledon")


[{"url": "https://www.sportingnews.com/uk/football/news/list-euros-host-nations-uefa-european-championship-countries/85f8069d69c9f4ecd00c4900", "content": "Complete list of Euros host nations: Countries to have held UEFA European Championship and how many times Germany has staged ahead of 2024 The 2024 UEFA European Championship, more commonly known as Euro 2024, will mark the 17th edition of the 
{
  "query": "euro 2024 host nation",
  "follow_up_questions": "None",
  "answer": "Germany is the host nation for Euro 2024, with matches taking place across ten cities: Berlin, Cologne, Munich, Frankfurt, Hamburg, Dortmund, Leipzig, Gelsenkirchen, Stuttgart, and D\u00fcsseldorf. The tou",
  "images": "['https://img.planetafobal.com/2021/10/sedes-uefa-euro-2024-alemania-fg.jpg', 'https://www.dfb.de/fileadmin/_processed_/202110/csm_248219-EURO2024_HC_Gelsenkirchen_FC_CMYK_266df023cc.png', 'https://th",
  "results": "[{'title': 'Complete list of Euros host nations: Countries to have held UEFA 

NotImplementedError: 